# Sentiment Analysis of Financial News Articles using FinBERT

In [1]:
import pandas as pd
import re
import torch

In [2]:
wsjFile = 'wsj_dataset.csv'
initial_df = pd.read_csv(wsjFile)

initial_df.head(10)

,date,title,softTitle,text,url,keywords,tags/0
0,2023-11-10,Marketing and Media News,Marketing and Media News,Bed and mattress brand Sleep Number said it re...,https://www.wsj.com/news/cmo-today?mod=nav_top...,"CMO, chief marketing officer, cmo news, chief ...",NaN
1,2023-07-10T21:22:00.000Z,What It's Like to Be Wrongfully Detained Overseas,What It's Like to Be Wrongfully Detained Overseas,What's News brings you the headlines and busin...,https://www.wsj.com/podcasts/whats-news/what-i...,"wall street journal columnists, analysis colum...",NaN
2,2023-11-10,Dow Opens Higher Ahead of Veteran's Day; Gloom...,Stock Market Today: Dow Opens Higher Ahead of ...,U. S. stocks are looking to end the week on a ...,https://www.wsj.com/livecoverage/stock-market-...,NaN,NaN
3,2023-11-08,"Israel Plans Humanitarian Pauses, U.S. Says, a...",Israel-Hamas War Live Updates: Israel Plans Hu...,Secretary of State Antony Blinken said that wh...,https://www.wsj.com/livecoverage/israel-hamas-...,NaN,NaN
4,2023-11-10T10:30:00.000Z,Why Are We So Obsessed with Sam Bankman-Fried’...,Why Are We So Obsessed with Sam Bankman-Fried’...,During the trial for disgraced crypto-wunderki...,https://www.wsj.com/us-news/law/why-are-we-so-...,"wsjreview,Financial Investment Services,Securi...",NaN
5,2023-07-10T21:22:00.000Z,What It's Like to Be Wrongfully Detained Overseas,What It's Like to Be Wrongfully Detained Overseas,What's News brings you the headlines and busin...,https://www.wsj.com/podcasts/whats-news/what-i...,"wall street journal columnists, analysis colum...",NaN
6,2023-11-10T16:00:00.000Z,"Two Minnesotans Retired in Cornwall, England. ...","Two Minnesotans Retired in Cornwall, England. ...",We didn’t move to Britain to undermine British...,https://www.wsj.com/lifestyle/travel/retiring-...,"journal reports,retirement,encore,encorereport...",NaN
7,2023-11-09T22:54:00.000Z,Brie Larson Leads a Not-So-Supergroup,‘The Marvels’ Review: Brie Larson Leads a Not-...,‘The Marvels” arrives amid a rumbling storm of...,https://www.wsj.com/arts-culture/film/the-marv...,"arts in review,Audiovisual Production,Media/En...",NaN
8,2023-11-08T15:52:00.000Z,Best Travel Credit Cards for 2024,Best Travel Credit Cards for 2024,Inflation is pushing up prices nearly everywhe...,https://www.wsj.com/buyside/personal-finance/b...,NaN,NaN
9,2023-11-10,Marketing and Media News,Marketing and Media News,Bed and mattress brand Sleep Number said it re...,https://www.wsj.com/news/cmo-today?mod=hp_mino...,"CMO, chief marketing officer, cmo news, chief ...",NaN


## Data Cleaning and Pre-processing

### Delete unwanted columns

In [3]:
#Remove tags and keywords column
df = initial_df.drop(columns=['tags/0', 'keywords'])
df.head(10)

,date,title,softTitle,text,url
0,2023-11-10,Marketing and Media News,Marketing and Media News,Bed and mattress brand Sleep Number said it re...,https://www.wsj.com/news/cmo-today?mod=nav_top...
1,2023-07-10T21:22:00.000Z,What It's Like to Be Wrongfully Detained Overseas,What It's Like to Be Wrongfully Detained Overseas,What's News brings you the headlines and busin...,https://www.wsj.com/podcasts/whats-news/what-i...
2,2023-11-10,Dow Opens Higher Ahead of Veteran's Day; Gloom...,Stock Market Today: Dow Opens Higher Ahead of ...,U. S. stocks are looking to end the week on a ...,https://www.wsj.com/livecoverage/stock-market-...
3,2023-11-08,"Israel Plans Humanitarian Pauses, U.S. Says, a...",Israel-Hamas War Live Updates: Israel Plans Hu...,Secretary of State Antony Blinken said that wh...,https://www.wsj.com/livecoverage/israel-hamas-...
4,2023-11-10T10:30:00.000Z,Why Are We So Obsessed with Sam Bankman-Fried’...,Why Are We So Obsessed with Sam Bankman-Fried’...,During the trial for disgraced crypto-wunderki...,https://www.wsj.com/us-news/law/why-are-we-so-...
5,2023-07-10T21:22:00.000Z,What It's Like to Be Wrongfully Detained Overseas,What It's Like to Be Wrongfully Detained Overseas,What's News brings you the headlines and busin...,https://www.wsj.com/podcasts/whats-news/what-i...
6,2023-11-10T16:00:00.000Z,"Two Minnesotans Retired in Cornwall, England. ...","Two Minnesotans Retired in Cornwall, England. ...",We didn’t move to Britain to undermine British...,https://www.wsj.com/lifestyle/travel/retiring-...
7,2023-11-09T22:54:00.000Z,Brie Larson Leads a Not-So-Supergroup,‘The Marvels’ Review: Brie Larson Leads a Not-...,‘The Marvels” arrives amid a rumbling storm of...,https://www.wsj.com/arts-culture/film/the-marv...
8,2023-11-08T15:52:00.000Z,Best Travel Credit Cards for 2024,Best Travel Credit Cards for 2024,Inflation is pushing up prices nearly everywhe...,https://www.wsj.com/buyside/personal-finance/b...
9,2023-11-10,Marketing and Media News,Marketing and Media News,Bed and mattress brand Sleep Number said it re...,https://www.wsj.com/news/cmo-today?mod=hp_mino...


In [4]:
#Handle missing values if any
missing_values = df.isnull().sum()
missing_values


date         0
title        0
softTitle    0
text         0
url          0
dtype: int64

In [5]:
# Generalise date time column
date_values = df['date'].values
date_values_updated = []

for value in date_values:
    if 'T' not in value:
        value += "T00:00:00.000Z"
    date_values_updated.append(value)

# Change the format of date to datetime for easy processing
df['date'] = date_values_updated
df.head()

,date,title,softTitle,text,url
0,2023-11-10T00:00:00.000Z,Marketing and Media News,Marketing and Media News,Bed and mattress brand Sleep Number said it re...,https://www.wsj.com/news/cmo-today?mod=nav_top...
1,2023-07-10T21:22:00.000Z,What It's Like to Be Wrongfully Detained Overseas,What It's Like to Be Wrongfully Detained Overseas,What's News brings you the headlines and busin...,https://www.wsj.com/podcasts/whats-news/what-i...
2,2023-11-10T00:00:00.000Z,Dow Opens Higher Ahead of Veteran's Day; Gloom...,Stock Market Today: Dow Opens Higher Ahead of ...,U. S. stocks are looking to end the week on a ...,https://www.wsj.com/livecoverage/stock-market-...
3,2023-11-08T00:00:00.000Z,"Israel Plans Humanitarian Pauses, U.S. Says, a...",Israel-Hamas War Live Updates: Israel Plans Hu...,Secretary of State Antony Blinken said that wh...,https://www.wsj.com/livecoverage/israel-hamas-...
4,2023-11-10T10:30:00.000Z,Why Are We So Obsessed with Sam Bankman-Fried’...,Why Are We So Obsessed with Sam Bankman-Fried’...,During the trial for disgraced crypto-wunderki...,https://www.wsj.com/us-news/law/why-are-we-so-...


In [6]:
# Remove special characters
def remove_special_characters(text):
    text = text.replace('\n', ' ')
    return re.sub(r'[^a-zA-Z\s]', '', text, re.I| re.A)

df['text'] = df['text'].apply(remove_special_characters)

df['text'].head()

0    Bed and mattress brand Sleep Number said it re...
1    Whats News brings you the headlines and busine...
2    U S stocks are looking to end the week on a hi...
3    Secretary of State Antony Blinken said that wh...
4    During the trial for disgraced cryptowunderkin...
Name: text, dtype: object

### Split the data into different dataframes based on different companies

In [7]:
# Companies mapping with their keywords
finalKeywordsMapping = {
    'JPM': ['JPM', 'Jamie Dimon', 'Jeremy Barnum', 'Lori Beer', 'Daniel E. Pinto', 'JPMorgan', 'JPMorgan Chase', 'JPMorgan Chase & Co', 'JPMorgan & Chase', 'Chase', 'Chase Sapphire Preferred', 'Chase Sapphire Reserve', 'Chase Freedom Flex', 'Chase Freedom Unlimited', 'Chase Slate Edge', 'Chase Ink Business Cards', 'First Republic Bank', 'Global Shares PLC', 'Frank Financial Aid', 'Volkswagen Payments SA', 'Nutmeg Saving and Investment Ltd', 'JPMorgan Coin', 'Onyx', 'Ownera'],
    'BAC': ['BAC', 'Brian Moynihan', 'Alastair Borthwick', 'Amadeo Giannini', 'Hugh McColl', 'Bank of America', 'Banc of America', 'Bank of America Private Bank', 'Balboa', 'BankAmericard', 'Alaska Airlines Visa', 'Alaska Airlines', 'Axia', 'AxiaMed', 'BACSIL'],
    'WFC': ['WFC', 'Charles W. Scharf', 'Scott Powell', 'Michael P. Santomassimo', 'William Fargo', 'Wells Fargo', 'Wells Fargo Autograph', 'Wells Fargo Active Cash', 'Wells Fargo Reflect', 'Bilt Mastercard', 'Choice Privileges', 'GTCR', 'Wells Fargo Digital Cash', 'Elliptic'],
    'C': ['C ', 'Jane Fraser', 'Michael Corbat', 'Citi', 'Citigroup', 'Citibank', 'Citi Double Cash', 'Citi Premier', 'Citi Custom Cash', 'Anywhere Visa', 'AAdvantage', 'Consumer Bank'],
    'GS': ['GS', 'David Solomon', 'Denis Coleman', 'Marco Argenti', 'John E. Waldron', 'Atte Lahtiranta', 'Goldman Sachs', 'Goldman', 'Sachs', 'Apple Card', 'Marcus', 'GreenSky', 'NN Investments', 'Next Capital', 'Ethereum Blockchain', 'Fnality', 'Metaverse'],
    'MS': ['MS', 'James Gorman', 'Sharon Yeshaya', 'Ted Pick', 'Andy Saperstein', 'Morgan Stanley', 'Stanley', 'Blue Cash', 'E trade', 'Eaton Vance', 'Figment', 'NYDIG', 'Defi'],
    'AXP': ['AXP', 'Stephen Squeri', 'John Butterfield', 'William Fargo', 'Henry Wells', 'Amex', 'American Express', 'Gold Card', 'Platinum Card', 'American Express Green Card', 'Delta SkyMiles', 'Hilton Honors', 'Marriott Bonvoy', 'Kabbage', 'Nipendo', 'Web3', 'Abra'],
    'V': ['V ', 'Ryan McInerney', 'Dee Hock', 'AI Kelly', 'Visa', 'Classic', 'Signature', 'Visa Platinum', 'Infinite', 'Currency cloud', 'Tink', 'Pismo', 'Visa B2B Connect', 'Wirex'],
    'MA': ['MA', 'Michael Miebach', 'Sachin Mehra', 'Raja Rajamannar', 'Ed McLaughlin', 'Mastercard', 'Standard Mastercard', 'World Elite', 'Gold Mastercard', 'Elite Mastercard', 'Finicity', 'Ekata', 'Aiia', 'CipherTrace', 'Arcus Financial Intelligence', 'Baffin Bay Networds', 'Dynamic Yield', 'Mastercard Crypto Credential', 'Aptos Labs', 'Ava Labs', 'Polygon', 'Solana'],
    'SCHW': ['SCHW', 'Walter W Bettinger II', 'Charles R. Schwab', 'Charles Schwab', 'Schwab', 'Schwab Investor', 'TD Ameritrade', 'Family Wealth Alliance', 'Wasmer', 'Schroeder', 'Schwab Crypto Thematic ETF', 'STCE']
}

finalKeywordsMapping


{'JPM': ['JPM',
  'Jamie Dimon',
  'Jeremy Barnum',
  'Lori Beer',
  'Daniel E. Pinto',
  'JPMorgan',
  'JPMorgan Chase',
  'JPMorgan Chase & Co',
  'JPMorgan & Chase',
  'Chase',
  'Chase Sapphire Preferred',
  'Chase Sapphire Reserve',
  'Chase Freedom Flex',
  'Chase Freedom Unlimited',
  'Chase Slate Edge',
  'Chase Ink Business Cards',
  'First Republic Bank',
  'Global Shares PLC',
  'Frank Financial Aid',
  'Volkswagen Payments SA',
  'Nutmeg Saving and Investment Ltd',
  'JPMorgan Coin',
  'Onyx',
  'Ownera'],
 'BAC': ['BAC',
  'Brian Moynihan',
  'Alastair Borthwick',
  'Amadeo Giannini',
  'Hugh McColl',
  'Bank of America',
  'Banc of America',
  'Bank of America Private Bank',
  'Balboa',
  'BankAmericard',
  'Alaska Airlines Visa',
  'Alaska Airlines',
  'Axia',
  'AxiaMed',
  'BACSIL'],
 'WFC': ['WFC',
  'Charles W. Scharf',
  'Scott Powell',
  'Michael P. Santomassimo',
  'William Fargo',
  'Wells Fargo',
  'Wells Fargo Autograph',
  'Wells Fargo Active Cash',
  'Wells F

In [8]:
# Filter out the dataframes
filteredDfs = {}

for key, keywords in finalKeywordsMapping.items():
    mask = df['text'].str.contains('|'.join(keywords), case=True, na=False)
    # Filtering the DataFrame based on the mask
    filtered_df = df[mask]

    # Removing duplicate rows based on the 'title' column
    filtered_df_unique = filtered_df.drop_duplicates(subset=['title'])

    # Storing the filtered and deduplicated DataFrame
    filteredDfs[key] = filtered_df_unique


filteredDfs

{'JPM':                          date  \
 8    2023-11-08T15:52:00.000Z   
 29   2022-06-20T13:38:00.000Z   
 30   2023-08-16T14:30:00.000Z   
 31   2022-07-03T14:01:00.000Z   
 33   2022-06-27T13:20:00.000Z   
 ..                        ...   
 818  2023-03-24T15:50:00.000Z   
 819  2023-02-22T14:58:00.000Z   
 865  2022-12-23T15:00:00.000Z   
 880  2023-04-23T12:00:00.000Z   
 948  2023-08-09T14:36:00.000Z   
 
                                                  title  \
 8                    Best Travel Credit Cards for 2024   
 29              If You Want One Travel Card…This Is It   
 30   How to Score Big Credit Card Rewards—Without P...   
 31                  The Best Travel Card for Beginners   
 33      The Ultimate Travel Card for Serious Travelers   
 ..                                                 ...   
 818                  The Best Mortgage Lenders of 2023   
 819  Your Credit Score Affects Your Mortgage Rate—H...   
 865  12 Best Gym Bags to Carry All Your Workout Ess

In [9]:
filteredDfs['JPM']['text'].head()

for key, item in filteredDfs.items():
    print(len(item), key)

62 JPM
27 BAC
34 WFC
187 C
22 GS
22 MS
45 AXP
173 V
31 MA
20 SCHW


## Applying FinBERT Algorithm 

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/Users/namanlalit/anaconda3/envs/sentimentAnalysis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def predictSentimentForCompany(title, newsArticleData):
    inputs = tokenizer(newsArticleData, padding = True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)

    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()

    table = {'title': title,
            "positive":positive,
            "negative":negative, 
            "neutral":neutral}
        
    df = pd.DataFrame(table, columns = ["title", "positive", "negative", "neutral"])
    return df

In [12]:
# Sentiment analysis for Wells Fargo
companyDf = predictSentimentForCompany(filteredDfs['WFC']['title'].astype(str).tolist(), filteredDfs['WFC']['text'].astype(str).tolist())
filteredDfs['WFC'] = pd.merge(filteredDfs['WFC'], companyDf, on='title', how='inner')

In [13]:
# Sentiment analysis for JP Morgan
companyDf = predictSentimentForCompany(filteredDfs['JPM']['title'].astype(str).tolist(), filteredDfs['JPM']['text'].astype(str).tolist())
filteredDfs['JPM'] = pd.merge(filteredDfs['JPM'], companyDf, on='title', how='inner')

In [14]:
# Sentiment analysis for Bank of America
companyDf = predictSentimentForCompany(filteredDfs['BAC']['title'].astype(str).tolist(), filteredDfs['BAC']['text'].astype(str).tolist())
filteredDfs['BAC'] = pd.merge(filteredDfs['BAC'], companyDf, on='title', how='inner')

In [15]:
# Sentiment analysis for Goldman Sachs
companyDf = predictSentimentForCompany(filteredDfs['GS']['title'].astype(str).tolist(), filteredDfs['GS']['text'].astype(str).tolist())
filteredDfs['GS'] = pd.merge(filteredDfs['GS'], companyDf, on='title', how='inner')

In [16]:
# Sentiment analysis for Citigroup
companyDf = predictSentimentForCompany(filteredDfs['C']['title'].astype(str).tolist(), filteredDfs['C']['text'].astype(str).tolist())
filteredDfs['C'] = pd.merge(filteredDfs['C'], companyDf, on='title', how='inner')

KeyboardInterrupt: 

In [17]:
filteredDfs['GS']

,date,title,softTitle,text,url,positive,negative,neutral
0,2023-11-08T21:53:00.000Z,The Best Online Banks for 2024,The Best Online Banks for 2024,More and more of Americans financial lives are...,https://www.wsj.com/buyside/personal-finance/b...,0.086377,0.015061,0.898561
1,2023-10-13T10:00:00.000Z,Going Electric? Why Future Power Could Come Fr...,Going Electric? Why Future Power Could Come Fr...,WSJs The Future of EverythingWhat will the fut...,https://www.wsj.com/podcasts/wsj-the-future-of...,0.050181,0.021606,0.928213
2,2022-11-28T20:50:00.000Z,The Best Cyber Monday Fashion Deals to Shop Now,The Best Cyber Monday Fashion Deals to Shop Now,The discounts reflected in these deals may no ...,https://www.wsj.com/buyside/shopping-holidays/...,0.043595,0.028076,0.928328
3,2022-11-10T15:59:00.000Z,Some People Are Getting 14 Times the Average S...,Some People Are Getting 14 Times the Average S...,After years of earning essentially zilch on yo...,https://www.wsj.com/buyside/personal-finance/t...,0.107108,0.017447,0.875446
4,2023-10-27T16:18:00.000Z,Best High-Yield Savings Accounts for November ...,Best High-Yield Savings Accounts for November ...,If you want to make sure that your savings are...,https://www.wsj.com/buyside/personal-finance/b...,0.091485,0.017460,0.891055
5,2023-10-29T10:00:00.000Z,The Best National Banks,The Best National Banks,For many of us the most important factor in ch...,https://www.wsj.com/buyside/personal-finance/b...,0.047398,0.022126,0.930475
6,2022-10-31T12:48:00.000Z,Best Store Credit Cards,Best Store Credit Cards,If you frequently shop at big retailers like A...,https://www.wsj.com/buyside/personal-finance/b...,0.059569,0.021387,0.919044
7,2023-03-16T18:08:00.000Z,The Best Bathrobes for Men and Women to Make M...,The Best Bathrobes for Men and Women to Make M...,A good bathrobe is a supremely practical loung...,https://www.wsj.com/buyside/style/best-robes-2...,0.076155,0.019731,0.904114
8,2023-09-20T13:33:00.000Z,The Best Office Chairs for a Comfortable Workd...,The Best Office Chairs for a Comfortable Workd...,Many health experts have adopted the mantra Si...,https://www.wsj.com/buyside/home/best-office-c...,0.151934,0.017659,0.830407
9,2023-10-17T17:10:00.000Z,"28 Gifts for Gardeners, According to Plant Exp...","28 Gifts for Gardeners, According to Plant Exp...",When picking out the perfect gardening gift it...,https://www.wsj.com/buyside/gifts/gifts-for-ga...,0.048260,0.026781,0.924960


In [ ]:
# Calculating the sentiment for each company
for key, item in filteredDfs.items():
    print(key)
    print(predictSentimentForCompany(item['title'], item['text'].astype(str).tolist()).head())

JPM


KeyboardInterrupt: 

In [ ]:
# JPM
lst = filteredDfs['JPM']['text'].astype(str).tolist()
inputs = tokenizer(lst, padding = True, truncation=True, return_tensors='pt')
print(inputs)

{'input_ids': tensor([[  101, 14200,  2003,  ...,  2070,  1997,   102],
        [  101,  3007,  2028,  ...,  2247,  2007,   102],
        [  101,  4923,  4003,  ..., 14550,  5643,   102],
        ...,
        [  101,  2017,  2089,  ...,  6978, 10047,   102],
        [  101,  5166,  2075,  ...,  2017,  2453,   102],
        [  101,  2122,  2420,  ...,  2038,  2815,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [ ]:
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([62, 3])


In [ ]:
import torch


predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)


tensor([[0.0561, 0.0231, 0.9208],
        [0.0417, 0.0373, 0.9210],
        [0.0520, 0.0228, 0.9252],
        [0.0581, 0.0214, 0.9205],
        [0.0365, 0.0479, 0.9156],
        [0.0490, 0.0248, 0.9263],
        [0.0629, 0.0333, 0.9038],
        [0.0573, 0.0244, 0.9183],
        [0.0515, 0.0247, 0.9238],
        [0.0559, 0.0204, 0.9237],
        [0.0631, 0.0219, 0.9150],
        [0.0846, 0.0175, 0.8979],
        [0.3289, 0.0175, 0.6537],
        [0.0756, 0.0169, 0.9075],
        [0.0282, 0.0572, 0.9147],
        [0.0915, 0.0175, 0.8911],
        [0.0453, 0.0229, 0.9319],
        [0.0619, 0.0268, 0.9114],
        [0.0474, 0.0221, 0.9305],
        [0.0546, 0.0207, 0.9247],
        [0.0384, 0.0315, 0.9301],
        [0.0596, 0.0214, 0.9190],
        [0.0404, 0.0318, 0.9278],
        [0.0379, 0.0364, 0.9258],
        [0.0329, 0.0580, 0.9092],
        [0.0361, 0.0484, 0.9155],
        [0.0508, 0.0281, 0.9211],
        [0.1288, 0.0318, 0.8394],
        [0.0329, 0.0696, 0.8975],
        [0.079

In [ ]:
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()


table = {'News Article': lst,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["News Article", "Positive", "Negative", "Neutral"])


df.head(62)

,News Article,Positive,Negative,Neutral
0,Inflation is pushing up prices nearly everywhe...,0.056134,0.023086,0.920779
1,Capital One Venture Rewards Credit Card offers...,0.041747,0.037252,0.921001
2,Credit card rewardswhen you do them rightcan f...,0.052007,0.022760,0.925233
3,The Chase Sapphire Preferred Card is a solid p...,0.058123,0.021357,0.920520
4,The Capital One Venture X Rewards Credit Card ...,0.036546,0.047877,0.915577
...,...,...,...,...
57,If youre in the market for a new home youve pr...,0.087703,0.016075,0.896222
58,If youve been tracking mortgage rates you know...,0.036469,0.044496,0.919035
59,You may work hard in the gym but dare we say y...,0.049456,0.026051,0.924493
60,Budgeting is hardly the funnest or most romant...,0.027070,0.073226,0.899704
